# Preliminaries

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
import sys
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import r2_score
from sklearn import preprocessing

#sys.path.append("/Users/paolo/Documents/methods/CMI_FS")
#from feature_selection import forwardFeatureSelection

sys.path.append("../LinCFA")
from LinCFA import LinCFA

sys.path.append("../NonLinCFA")
from NonLinCFA import NonLinCFA

sys.path.append("../GenLinCFA")
from GenLinCFA import GenLinCFA

sys.path.append("../droughts")
from aux import prepare_target,prepare_features,compare_methods

#from aux import standardize,unfold_dataset,compute_r2,prepare_target,prepare_features,aggregate_unfolded_data,aggregate_unfolded_data_onlyTrain,FS_with_linearWrapper,compare_methods, compute_r2


In [2]:
def compute_CI(list,n):
    print(f'{np.mean(list)} +- {1.96*np.std(list)/np.sqrt(n)}')

In [3]:
from sklearn.base import TransformerMixin, BaseEstimator
from sklearn import utils
from sklearn.preprocessing import KernelCenterer, scale
from sklearn.metrics.pairwise import pairwise_kernels
from scipy import linalg
from scipy.sparse.linalg import eigsh as ssl_eigsh
from scipy.io import arff
from sklearn.impute import SimpleImputer

class spca(BaseEstimator, TransformerMixin):
    
    def __init__(self, num_components, kernel="linear", eigen_solver='auto', 
                 max_iterations=None, gamma=0, degree=3, coef0=1, alpha=1.0, 
                 tolerance=0, fit_inverse_transform=False):
        
        self._num_components = num_components
        self._gamma = gamma
        self._tolerance = tolerance
        self._fit_inverse_transform = fit_inverse_transform
        self._max_iterations = max_iterations
        self._degree = degree
        self._kernel = kernel
        self._eigen_solver = eigen_solver
        self._coef0 = coef0
        self._centerer = KernelCenterer()
        self._alpha = alpha
        self._alphas = []
        self._lambdas = []
        
        
    def _get_kernel(self, X, Y=None):
        # Returns a kernel matrix K such that K_{i, j} is the kernel between the ith and jth vectors 
        # of the given matrix X, if Y is None. 
        
        # If Y is not None, then K_{i, j} is the kernel between the ith array from X and the jth array from Y.
        
        # valid kernels are 'linear, rbf, poly, sigmoid, precomputed'
        
        args = {"gamma": self._gamma, "degree": self._degree, "coef0": self._coef0}
        
        return pairwise_kernels(X, Y, metric=self._kernel, n_jobs=-1, filter_params=True, **args)
    
    
    
    def _fit(self, X, Y):
        
        # calculate kernel matrix of the labels Y and centre it and call it K (=H.L.H)
        K = self._centerer.fit_transform(self._get_kernel(Y))
        
        # deciding on the number of components to use
        if self._num_components is not None:
            num_components = min(K.shape[0], self._num_components)
        else:
            num_components = self.K.shape[0]
        
        # Scale X
        # scaled_X = scale(X)
        
        # calculate the eigen values and eigen vectors for X^T.K.X
        Q = (X.T).dot(K).dot(X)
        
        # If n_components is much less than the number of training samples, 
        # arpack may be more efficient than the dense eigensolver.
        if (self._eigen_solver=='auto'):
            if (Q.shape[0]/num_components) > 20:
                eigen_solver = 'arpack'
            else:
                eigen_solver = 'dense'
        else:
            eigen_solver = self._eigen_solver
        
        if eigen_solver == 'dense':
            # Return the eigenvalues (in ascending order) and eigenvectors of a Hermitian or symmetric matrix.
            self._lambdas, self._alphas = linalg.eigh(Q, eigvals=(Q.shape[0] - num_components, Q.shape[0] - 1))
            # argument eigvals = Indexes of the smallest and largest (in ascending order) eigenvalues
        
        elif eigen_solver == 'arpack':
            # deprecated :: self._lambdas, self._alphas = utils.arpack.eigsh(A=Q, num_components, which="LA", tol=self._tolerance)
            self._lambdas, self._alphas = ssl_eigsh(A=Q, k=num_components, which="LA", tol=self._tolerance)
            
        indices = self._lambdas.argsort()[::-1]
        
        self._lambdas = self._lambdas[indices]
        self._lambdas = self._lambdas[self._lambdas > 0]  # selecting values only for non zero eigen values
        
        self._alphas = self._alphas[:, indices]
        #return self._alphas
        self._alphas = self._alphas[:, self._lambdas > 0]  # selecting values only for non zero eigen values
        
        self.X_fit = X

        
    def _transform(self):
        return self.X_fit.dot(self._alphas)
        
        
    def transform(self, X):
        return X.dot(self._alphas)
        
        
    def fit(self, X, Y):
        self._fit(X,Y)
        return
        
        
    def fit_and_transform(self, X, Y):
        self.fit(X, Y)
        return self._transform()
    

In [16]:
data = arff.loadarff('/Users/paolo/Documents/PaperNon_Gen_LinCFA/benchmarks/PolishBankruptcy/1year.arff')
df= pd.DataFrame(data[0])
df['class']=df.apply(lambda x:1 if x['class']==b'1' else 0,axis=1)
df = df.sample(frac=1)
imp = SimpleImputer(missing_values=np.nan, strategy='mean')
df = pd.DataFrame(imp.fit_transform(df),columns=df.columns)
df = df.dropna()
normalized_df=(df-df.mean())/df.std()
normalized_df['class']=df['class']
normalized_df['mean_std'] = normalized_df['class']
normalized_df = normalized_df.drop(['class'],axis=1)
normalized_df = pd.concat((normalized_df[normalized_df.mean_std==0].sample(813),normalized_df[normalized_df.mean_std==1])).sample(frac=1).reset_index(drop=True)
normalized_df 

,Attr1,Attr2,Attr3,Attr4,Attr5,Attr6,Attr7,Attr8,Attr9,Attr10,...,Attr56,Attr57,Attr58,Attr59,Attr60,Attr61,Attr62,Attr63,Attr64,mean_std
0,-0.007485,-0.042269,-0.000682,-0.087924,0.006503,-0.009870,-0.036964,-0.033794,-0.046992,-0.034288,...,0.011942,-0.044317,-0.011937,-0.041967,-0.016505,-0.044070,-0.014941,-0.254707,-0.040468,1.0
1,0.000453,0.067840,0.006428,-0.066720,0.014883,-0.009870,-0.033185,-0.136047,-0.028923,-0.051699,...,0.011950,0.065452,-0.011947,1.422417,-0.014263,-0.024062,-0.015233,0.225630,-0.039909,0.0
2,0.010936,-0.012093,0.019699,-0.065594,0.006324,0.004110,-0.024036,-0.085992,-0.043245,-0.039060,...,0.011940,-0.005890,-0.011940,0.019190,-0.016554,-0.031183,-0.015089,-0.159397,-0.040114,0.0
3,0.054435,-0.052669,0.022663,-0.037096,0.010021,-0.009870,0.004371,-0.001778,-0.045357,-0.032644,...,0.011959,0.045854,-0.011959,-0.028941,-0.016219,-0.040890,-0.015060,-0.186464,-0.040374,0.0
4,-0.002551,0.053346,-0.064531,-0.153015,0.176646,-0.010859,-0.034246,-0.130728,-0.044938,-0.049407,...,0.011928,-0.010176,-0.011924,0.167427,-0.016412,-0.042594,-0.014651,-0.320164,-0.040417,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1079,-0.006555,-0.051751,0.009012,-0.074173,0.007037,-0.006970,-0.037153,-0.015100,-0.044477,-0.034352,...,0.011932,-0.042844,-0.011932,-0.037915,-0.016384,-0.027171,-0.015134,-0.102169,-0.040168,0.0
1080,-0.013652,-0.055018,0.044609,0.002045,0.007894,-0.009870,-0.041555,0.007304,-0.046399,-0.032273,...,0.011927,-0.053162,-0.011928,-0.043831,-0.016614,-0.036972,-0.015019,-0.215547,-0.040330,0.0
1081,0.054185,-0.064374,0.000702,-0.077900,0.007306,0.036741,-0.003818,0.051224,-0.043107,-0.031123,...,0.011941,0.039498,-0.011941,-0.041938,-0.015719,-0.012324,-0.015213,0.110012,-0.040081,0.0
1082,0.018219,0.046404,0.014023,-0.104826,-0.000961,-0.009870,-0.019555,-0.127819,-0.041653,-0.048309,...,0.011964,0.097102,-0.011964,-0.043831,-0.016665,-0.021510,-0.014606,-0.325557,0.001265,1.0


In [17]:
df_trainVal_withTar = normalized_df.iloc[:-358,:]
df_test_withTar = normalized_df.iloc[-358:,:]
df_trainVal = normalized_df.iloc[:-358,:-1]
df_test = normalized_df.iloc[-358:,:-1]
target_df_trainVal = normalized_df.iloc[:-358,-1]
target_df_test = normalized_df.iloc[-358:,-1]

# Classification

# GenLinCFA

In [18]:
for eps in [0.06,0.07,0.08,0.09,0.1]:
    res = []
    nums = []
    for curr_seed in [0,1,2,3,4]:
        curr_df_trainVal = df_trainVal[np.random.default_rng(seed=curr_seed).permutation(df_trainVal.columns.values)]
        curr_df_test = df_test[np.random.default_rng(seed=curr_seed).permutation(df_test.columns.values)]
        curr_df_trainVal_withTar = pd.concat((curr_df_trainVal,target_df_trainVal), axis=1)
        
        output = GenLinCFA(curr_df_trainVal_withTar,'mean_std', eps, -5 , 0, 1).compute_clusters()
        
        aggregate_trainVal = pd.DataFrame()
        aggregate_test = pd.DataFrame()
        for i in range(len(output)):
            aggregate_trainVal[str(i)] = curr_df_trainVal_withTar[output[i]].mean(axis=1)
            aggregate_trainVal = aggregate_trainVal.copy()
            aggregate_test[str(i)] = curr_df_test[output[i]].mean(axis=1)
            aggregate_test = aggregate_test.copy()
        print(f'Number of aggregated features: {len(output)}\n')
        mod = LogisticRegression().fit(aggregate_trainVal, df_trainVal_withTar.mean_std)
        score = mod.score(aggregate_test, df_test_withTar.mean_std)
        print(score)
        res.append(score)
        nums.append(len(output))
    compute_CI(res,5)
    compute_CI(nums,5)


2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
24
1
1
1
1
2
3
4
5
6
7
8
9
10
11
12
12
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
Number of aggregated features: 30

0.7486033519553073
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
23
1
2
3
4
5
6
7
8
9
10
10
1
1
1
1
1
1
2
3
4
4
1
1
1
1
1
1
2
2
1
1
1
1
1
1
1
1
1
1
1
1
Number of aggregated features: 29

0.7513966480446927
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
24
1
1
1
1
1
1
1
1
2
3
4
5
6
7
8
8
1
1
2
2
1
1
1
1
1
2
2
1
1
1
1
1
1
1
1
1
1
1
1
1
Number of aggregated features: 32

0.7513966480446927
2
3
3
1
2
2
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
47
1
1
1
1
1
1
1
1
1
1
1
Number of aggregated features: 15

0.7513966480446927
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
22
1
2
3
4
5
6
7
8
8
1
2
3
4
4
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
2
1
1
1
1
1
1
1
1
1
1
Number of aggregated features: 32

0.7

In [ ]:
0.7502793296089385 +- 0.0011994818019107655
27.6 +- 5.6153230361217865

0.7480446927374301 +- 0.0018322387173260362
16.4 +- 3.444250397401443

0.7430167597765364 +- 0.00437988826815642
9.8 +- 1.5080541104350333

0.741340782122905 +- 0.0024969204840159806
8.2 +- 0.350615458871967

0.740782122905028 +- 0.002398963603821495
7.0 +- 0.5543717164502532

# Baselines

## pca

In [20]:
# explain 95% of variance
from sklearn.decomposition import PCA
from sklearn.utils import resample

acc = []
n = []
for curr_seed in [0,1,2,3,4]: 
    curr_df_trainVal_withTar = resample(df_trainVal_withTar, random_state=curr_seed)
    curr_df_trainVal = curr_df_trainVal_withTar.iloc[:,:-1]
    pca = PCA(n_components=0.95)
    trainVal_pca = pd.DataFrame(pca.fit_transform(curr_df_trainVal))
    test_pca = pd.DataFrame(pca.transform(df_test))
    mod = LogisticRegression().fit(trainVal_pca, curr_df_trainVal_withTar.mean_std)
    score = mod.score(test_pca, df_test_withTar.mean_std)
    acc.append(score)
    print(test_pca.shape[1])
    n.append(test_pca.shape[1])
compute_CI(acc,5)
compute_CI(n,5)
    

3
6
5
5
5
0.741340782122905 +- 0.003664477434652039
4.8 +- 0.8588289701681003


In [ ]:
0.741340782122905 +- 0.003664477434652039
4.8 +- 0.8588289701681003

## kernel pca

In [19]:
from sklearn.decomposition import KernelPCA
for curr_kernel in ['linear', 'poly', 'sigmoid']:
    best_scores = []
    best_nums = []
    for curr_seed in [0,1,2,3,4]: 
        best_score = 0
        best_num = 0
        print(f'#########################{curr_kernel}, {curr_seed}#############################')
        for i in range(50):
            curr_df_trainVal_withTar = resample(df_trainVal_withTar, random_state=curr_seed)
            curr_df_trainVal = curr_df_trainVal_withTar.iloc[:,:-1]
            dimRedMethod = KernelPCA(n_components=i+1, kernel=curr_kernel)
            trainVal_reduced = pd.DataFrame(dimRedMethod.fit_transform(curr_df_trainVal))
            test_reduced = pd.DataFrame(dimRedMethod.transform(df_test))
            print(test_reduced.shape)
            mod = LogisticRegression().fit(trainVal_reduced, curr_df_trainVal_withTar.mean_std)
            actual_score = mod.score(test_reduced, df_test_withTar.mean_std)
            print(actual_score)
            if actual_score> best_score:
                best_score=actual_score
                best_num = test_reduced.shape
        print(best_score)
        print(best_num)
        best_scores.append(best_score)
        best_nums.append(best_num[1])
    compute_CI(best_scores,5)
    compute_CI(best_nums,5)

#########################linear, 0#############################
(358, 1)
0.7430167597765364
(358, 2)
0.7458100558659218
(358, 3)
0.7486033519553073
(358, 4)
0.7569832402234636
(358, 5)
0.7597765363128491
(358, 6)
0.7597765363128491
(358, 7)
0.7681564245810056
(358, 8)
0.7681564245810056
(358, 9)
0.7681564245810056
(358, 10)
0.7681564245810056
(358, 11)
0.770949720670391
(358, 12)
0.770949720670391
(358, 13)
0.7681564245810056
(358, 14)
0.7681564245810056
(358, 15)
0.7681564245810056
(358, 16)
0.7681564245810056
(358, 17)
0.7681564245810056
(358, 18)
0.7681564245810056
(358, 19)
0.7681564245810056
(358, 20)
0.7653631284916201
(358, 21)
0.770949720670391
(358, 22)
0.770949720670391
(358, 23)
0.7681564245810056
(358, 24)
0.7681564245810056
(358, 25)
0.7653631284916201
(358, 26)
0.7653631284916201
(358, 27)
0.7653631284916201
(358, 28)
0.7653631284916201
(358, 29)
0.7653631284916201
(358, 30)
0.7653631284916201
(358, 31)
0.7653631284916201
(358, 32)
0.7653631284916201
(358, 33)
0.765363128

/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stab

(358, 10)
0.7374301675977654
(358, 11)
0.7374301675977654
(358, 12)
0.7374301675977654
(358, 13)


/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stab

0.7374301675977654
(358, 14)
0.7374301675977654
(358, 15)
0.7374301675977654


...

0.7486033519553073
(358, 38)
0.7486033519553073
(358, 39)
0.7486033519553073
(358, 40)


/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stab

0.7486033519553073
(358, 41)
0.7486033519553073
(358, 42)
0.7486033519553073


/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stab

(358, 43)
0.7486033519553073
(358, 44)
0.7486033519553073
(358, 45)
0.7486033519553073
(358, 46)
0.7486033519553073
(358, 47)
0.7486033519553073
(358, 48)
0.7486033519553073
(358, 49)
0.7486033519553073
(358, 50)
0.7486033519553073
0.7486033519553073
(358, 15)
0.7441340782122905 +- 0.0024969204840159694
13.6 +- 6.825754522395307
#########################sigmoid, 0#############################
(358, 1)
0.7402234636871509
(358, 2)
0.7402234636871509
(358, 3)
0.7402234636871509
(358, 4)
0.7402234636871509
(358, 5)
0.7402234636871509
(358, 6)
0.7402234636871509
(358, 7)
0.7402234636871509
(358, 8)
0.7402234636871509
(358, 9)
0.7402234636871509
(358, 10)
0.7402234636871509
(358, 11)
0.7402234636871509
(358, 12)
0.7402234636871509
(358, 13)
0.7402234636871509
(358, 14)
0.7402234636871509
(358, 15)
0.7402234636871509
(358, 16)
0.7402234636871509
(358, 17)
0.7402234636871509
(358, 18)
0.7402234636871509
(358, 19)
0.7402234636871509
(358, 20)
0.7402234636871509
(358, 21)
0.7402234636871509
(358

In [ ]:
0.1317877094972 +- 0.006076848351732443
13.8 +- 5.062722113645978

0.7441340782122905 +- 0.0024969204840159694
13.6 +- 6.825754522395307

0.7402234636871509 +- 0.0
1.0 +- 0.0

## isomap

In [21]:
from sklearn.manifold import Isomap
best_scores = []
best_nums = []
for curr_seed in [0,1,2,3,4]: 
    best_score = 0
    best_num = 0
    for i in range(50):
        curr_df_trainVal_withTar = resample(df_trainVal_withTar, random_state=curr_seed)
        curr_df_trainVal = curr_df_trainVal_withTar.iloc[:,:-1]
        dimRedMethod = Isomap(n_components=i+1,n_neighbors=10)
        trainVal_reduced = pd.DataFrame(dimRedMethod.fit_transform(curr_df_trainVal.values,curr_df_trainVal_withTar.mean_std.values.reshape(-1, 1)))
        test_reduced = pd.DataFrame(dimRedMethod.transform(df_test))
        print(test_reduced.shape)
        mod = LogisticRegression().fit(trainVal_reduced, curr_df_trainVal_withTar.mean_std)
        actual_score = mod.score(test_reduced, df_test_withTar.mean_std)
        print(actual_score)
        if actual_score> best_score:
            best_score=actual_score
            best_num = test_reduced.shape[1]
    print(best_score)
    print(best_num)
    best_scores.append(best_score)
    best_nums.append(best_num)
compute_CI(best_scores,5)
compute_CI(best_nums,5)

/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but NearestNeighbors was fitted without feature names
  warnings.warn(
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but NearestNeighbors was fitted without feature names
  warnings.warn(


(358, 1)
0.7402234636871509
(358, 2)
0.7402234636871509


/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but NearestNeighbors was fitted without feature names
  warnings.warn(
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but NearestNeighbors was fitted without feature names
  warnings.warn(


(358, 3)
0.7402234636871509
(358, 4)
0.7374301675977654


/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but NearestNeighbors was fitted without feature names
  warnings.warn(
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but NearestNeighbors was fitted without feature names
  warnings.warn(


...

/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but NearestNeighbors was fitted without feature names
  warnings.warn(


(358, 48)
0.7430167597765364


/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but NearestNeighbors was fitted without feature names
  warnings.warn(


(358, 49)
0.7402234636871509
(358, 50)
0.7374301675977654
0.7458100558659218
22
0.7463687150837989 +- 0.0039174911605806455
21.2 +- 9.814101853965036


/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but NearestNeighbors was fitted without feature names
  warnings.warn(


In [ ]:
0.7463687150837989 +- 0.0039174911605806455
21.2 +- 9.814101853965036

## LLE

In [22]:
from sklearn.manifold import LocallyLinearEmbedding as LLE

best_scores = []
best_nums = []
for curr_seed in [0,1,2,3,4]: 
    best_score = 0
    best_num = 0
    for i in range(50):
        curr_df_trainVal_withTar = resample(df_trainVal_withTar, random_state=curr_seed)
        curr_df_trainVal = curr_df_trainVal_withTar.iloc[:,:-1]
        dimRedMethod = LLE(n_components=i+1,n_neighbors=10)
        trainVal_reduced = pd.DataFrame(dimRedMethod.fit_transform(curr_df_trainVal.values,curr_df_trainVal_withTar.mean_std.values.reshape(-1, 1)))
        test_reduced = pd.DataFrame(dimRedMethod.transform(df_test))
        print(test_reduced.shape)
        mod = LogisticRegression().fit(trainVal_reduced, curr_df_trainVal_withTar.mean_std)
        actual_score = mod.score(test_reduced, df_test_withTar.mean_std)
        print(actual_score)
        if actual_score> best_score:
            best_score=actual_score
            best_num = test_reduced.shape[1]
    print(best_score)
    print(best_num)
    best_scores.append(best_score)
    best_nums.append(best_num)
compute_CI(best_scores,5)
compute_CI(best_nums,5)

/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but LocallyLinearEmbedding was fitted without feature names
  warnings.warn(
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but LocallyLinearEmbedding was fitted without feature names
  warnings.warn(
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but LocallyLinearEmbedding was fitted without feature names
  warnings.warn(
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but LocallyLinearEmbedding was fitted without feature names
  warnings.warn(


(358, 1)
0.7402234636871509
(358, 2)
0.7402234636871509
(358, 3)
0.7402234636871509
(358, 4)
0.7402234636871509


/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but LocallyLinearEmbedding was fitted without feature names
  warnings.warn(
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but LocallyLinearEmbedding was fitted without feature names
  warnings.warn(
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but LocallyLinearEmbedding was fitted without feature names
  warnings.warn(
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but LocallyLinearEmbedding was fitted without feature names
  warnings.warn(


(358, 5)
0.7402234636871509
(358, 6)
0.7402234636871509
(358, 7)
0.7402234636871509
(358, 8)
0.7402234636871509


/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but LocallyLinearEmbedding was fitted without feature names
  warnings.warn(
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but LocallyLinearEmbedding was fitted without feature names
  warnings.warn(
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but LocallyLinearEmbedding was fitted without feature names
  warnings.warn(


...

/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but LocallyLinearEmbedding was fitted without feature names
  warnings.warn(
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but LocallyLinearEmbedding was fitted without feature names
  warnings.warn(
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but LocallyLinearEmbedding was fitted without feature names
  warnings.warn(


(358, 44)
0.7402234636871509
(358, 45)
0.7402234636871509
(358, 46)
0.7402234636871509


/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but LocallyLinearEmbedding was fitted without feature names
  warnings.warn(
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but LocallyLinearEmbedding was fitted without feature names
  warnings.warn(
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but LocallyLinearEmbedding was fitted without feature names
  warnings.warn(


(358, 47)
0.7402234636871509
(358, 48)
0.7402234636871509
(358, 49)
0.7402234636871509
(358, 50)
0.7402234636871509
0.7402234636871509
1
0.7402234636871509 +- 0.0
1.0 +- 0.0


/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but LocallyLinearEmbedding was fitted without feature names
  warnings.warn(


In [ ]:
0.7402234636871509 +- 0.0
1.0 +- 0.0

## Sup PCA

In [76]:
for kernel in ['sigmoid']: #['sigmoid']:#['poly']:#['linear']:
    best_scores = []
    best_nums = []
    for curr_seed in [5,2,13,6,5]: 
        print(f'#########################{kernel}, {curr_seed}#############################')
        best_score = 0
        best_num = 0
        for i in range(45):
            curr_df_trainVal_withTar = resample(df_trainVal_withTar, random_state=curr_seed)
            curr_df_trainVal = curr_df_trainVal_withTar.iloc[:,:-1]
            dimRedMethod = spca(num_components=i+1, kernel=kernel, degree=3, gamma=None, coef0=1)
            trainVal_reduced = pd.DataFrame(dimRedMethod.fit_and_transform(curr_df_trainVal.values,curr_df_trainVal_withTar.mean_std.values.reshape(-1, 1)))
            test_reduced = pd.DataFrame(dimRedMethod.transform(df_test))
            print(test_reduced.shape)
            mod = LogisticRegression().fit(trainVal_reduced, curr_df_trainVal_withTar.mean_std)
            actual_score = mod.score(test_reduced, df_test_withTar.mean_std)
            print(actual_score)
            if actual_score> best_score:
                best_score=actual_score
                best_num = test_reduced.shape
        print(best_score)
        print(best_num)
        best_scores.append(best_score)
        best_nums.append(best_num[1])
    compute_CI(best_scores,5)
    compute_CI(best_nums,5)

#########################sigmoid, 5#############################
(358, 1)
0.7374301675977654
(358, 2)
0.7318435754189944
(358, 3)
0.7374301675977654
(358, 4)
0.7374301675977654
(358, 5)
0.7374301675977654
(358, 6)
0.7374301675977654
(358, 7)
0.7402234636871509
(358, 8)
0.7374301675977654
(358, 9)
0.7346368715083799
(358, 10)
0.7402234636871509
(358, 11)
0.7374301675977654
(358, 12)
0.7374301675977654
(358, 13)
0.7430167597765364
(358, 14)
0.7374301675977654
(358, 15)
0.7402234636871509
(358, 16)
0.7402234636871509
(358, 17)
0.7402234636871509
(358, 18)
0.7402234636871509
(358, 19)
0.7402234636871509
(358, 20)
0.7318435754189944
(358, 21)
0.7402234636871509
(358, 22)
0.7402234636871509
(358, 23)
0.7346368715083799
(358, 24)
0.7346368715083799
(358, 25)
0.7402234636871509
(358, 26)
0.7374301675977654
(358, 27)
0.7402234636871509
(358, 28)
0.7374301675977654
(358, 29)
0.7346368715083799
(358, 30)
0.7374301675977654
(358, 31)
0.7318435754189944
(358, 32)
0.7346368715083799
(358, 33)
0.7318

In [ ]:
0.7508379888268156 +- 0.0018322387173260362
16.2 +- 7.528033156143774

0.7486033519553073 +- 0.0015485243476264195
18.6 +- 12.378761682817872

0.747486033519553 +- 0.003664477434652031
18.2 +- 9.365437053336057

## NDA

In [77]:
from sklearn.neighbors import NeighborhoodComponentsAnalysis as NCA

best_scores = []
best_nums = []
for curr_seed in [0,1,2,3,4]: 
    best_score = 0
    best_num = 0
    for i in range(50):
        curr_df_trainVal_withTar = resample(df_trainVal_withTar, random_state=curr_seed)
        curr_df_trainVal = curr_df_trainVal_withTar.iloc[:,:-1]
        dimRedMethod = NCA(n_components=i+1)
        trainVal_reduced = pd.DataFrame(dimRedMethod.fit_transform(curr_df_trainVal.values,curr_df_trainVal_withTar.mean_std.values.reshape(-1, 1)))
        test_reduced = pd.DataFrame(dimRedMethod.transform(df_test))
        print(test_reduced.shape)
        mod = LogisticRegression(random_state=0).fit(trainVal_reduced, curr_df_trainVal_withTar.mean_std)
        actual_score = mod.score(test_reduced, df_test_withTar.mean_std)
        print(actual_score)
        if actual_score> best_score:
            best_score=actual_score
            best_num = test_reduced.shape
    print(best_score)
    print(best_num)
    best_scores.append(best_score)
    best_nums.append(best_num[1])
compute_CI(best_scores,5)
compute_CI(best_nums,5)


/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but NeighborhoodComponentsAnalysis was fitted without feature names
  warnings.warn(
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


(358, 1)
0.7681564245810056


/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but NeighborhoodComponentsAnalysis was fitted without feature names
  warnings.warn(
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


(358, 2)
0.7402234636871509


/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but NeighborhoodComponentsAnalysis was fitted without feature names
  warnings.warn(
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


...

/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but NeighborhoodComponentsAnalysis was fitted without feature names
  warnings.warn(
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for exampl

(358, 48)
0.7458100558659218


/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but NeighborhoodComponentsAnalysis was fitted without feature names
  warnings.warn(
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for exampl

(358, 49)
0.7346368715083799
(358, 50)
0.7346368715083799
0.7541899441340782
(358, 17)
0.7636871508379889 +- 0.007986543409266725
12.0 +- 9.537771647507606


/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but NeighborhoodComponentsAnalysis was fitted without feature names
  warnings.warn(
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
0.7636871508379889 +- 0.007986543409266725
12.0 +- 9.537771647507606

## lda

In [81]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
acc = []
for curr_seed in [0,1,2,3,4]: 
    for i in range(1):
        curr_df_trainVal_withTar = resample(df_trainVal_withTar, random_state=curr_seed)
        curr_df_trainVal = curr_df_trainVal_withTar.iloc[:,:-1]
        dimRedMethod = LDA(n_components=i+1)
        trainVal_reduced = pd.DataFrame(dimRedMethod.fit_transform(curr_df_trainVal,curr_df_trainVal_withTar.mean_std))
        test_reduced = pd.DataFrame(dimRedMethod.transform(df_test))
        print(test_reduced.shape)
        mod = LogisticRegression(random_state=0).fit(trainVal_reduced, curr_df_trainVal_withTar.mean_std)
        acc.append(mod.score(test_reduced, df_test_withTar.mean_std))
    print(acc)
compute_CI(acc,5)
        

(358, 1)
[0.7597765363128491]
(358, 1)
[0.7597765363128491, 0.770949720670391]
(358, 1)
[0.7597765363128491, 0.770949720670391, 0.7597765363128491]
(358, 1)
[0.7597765363128491, 0.770949720670391, 0.7597765363128491, 0.7486033519553073]
(358, 1)
[0.7597765363128491, 0.770949720670391, 0.7597765363128491, 0.7486033519553073, 0.7541899441340782]
0.758659217877095 +- 0.006496424149589931
